### Initialization & load initial events and items

In [1]:
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# %matplotlib inline
# %config InlineBackend.figure_format = 'png'
# %config InlineBackend.figure_format = 'retina'

In [3]:
items = pd.read_parquet('goodsread/items.parquet')
events = pd.read_parquet('goodsread/events.parquet')

3.2 Gb

### Тема 2 Урок 4 Холодный старт

Задание 2 из 6     
Завершите код для разбиения всех событий. В качестве точки разбиения используйте 2017-08-01, то есть отнесите в тестовую часть три последних месяца.

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime('2017-08-01').date()

train_test_global_time_split_idx = (
    events["started_at"] < train_test_global_time_split_date
)

events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train['user_id'].drop_duplicates()
users_test = events_test['user_id'].drop_duplicates()

# количество пользователей, которые есть и в train, и в test
common_users = set(users_train) & set(users_test)

print(len(users_train), len(users_test), len(common_users))

# Приборка
del train_test_global_time_split_idx, train_test_global_time_split_date, events

428220 123223 120858


Задание 3 из 6    
Идентифицируйте холодных пользователей и оцените их количество.

In [5]:
cold_users = set(users_test) - common_users

print(len(cold_users))

2365


Задание 4 из 6   
Завершите код, чтобы получить топ-100 наиболее популярных книг согласно условиям выше.

In [6]:
top_pop_start_date = pd.to_datetime('2015-01-01').date()

item_popularity = (
    events_train # type: ignore
    .query('started_at >= @top_pop_start_date')
    .groupby(['item_id'])
    .agg(users=('user_id', 'nunique'), avg_rating=('rating', 'mean'))
    .reset_index()
)
item_popularity['popularity_weighted'] = (
    item_popularity['users'] * item_popularity['avg_rating']
)

del top_pop_start_date
item_popularity.head(3)

,item_id,users,avg_rating,popularity_weighted
0,1,8728,4.796059,41860.0
1,2,9528,4.685873,44647.0
2,3,15139,4.706057,71245.0


In [7]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values(
    'popularity_weighted',
    ascending=False
)
# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = (
    item_popularity
    .query('avg_rating >= 4')
    .sort_values('popularity_weighted', ascending=False)
    .head(100)
)
top_k_pop_items.head(3)

,item_id,users,avg_rating,popularity_weighted
32387,18007564,20207,4.321275,87320.0
32623,18143977,19462,4.290669,83505.0
30695,16096824,16770,4.301014,72128.0


In [8]:
# добавляем информацию о книгах
if not set(top_k_pop_items.columns) & {
    "author", "title", "genre_and_votes", "publication_year"
}:
    top_k_pop_items = top_k_pop_items.merge(
        items.set_index("item_id")[
            ["author", "title", "genre_and_votes", "publication_year"]
        ],
        on="item_id",
        how='left'
    )
top_k_pop_items.head(3)

,item_id,users,avg_rating,popularity_weighted,author,title,genre_and_votes,publication_year
0,18007564,20207,4.321275,87320.0,Andy Weir,The Martian,"{'Science Fiction': 11966, 'Fiction': 8430}",2014
1,18143977,19462,4.290669,83505.0,Anthony Doerr,All the Light We Cannot See,"{'Historical-Historical Fiction': 13679, 'Fict...",2014
2,16096824,16770,4.301014,72128.0,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman...",2015


Задание 5 из 6    
Завершите предлагаемый код, чтобы в cold_users_events_with_recs для каждого события получить столбец avg_rating. В нём при совпадении по item_id будет значение из одноимённого столбца из top_k_pop_items, иначе — пропуск.
В cold_users_events_with_recs после выполнения завершённого кода должно быть столько же строк, сколько было до его выполнения.

In [9]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[['item_id', 'avg_rating']], on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]] 

In [10]:
len(cold_user_recs) / len(cold_users_events_with_recs)

0.19768403639371382

Задание 6    
Посчитайте метрики rmse и mae для полученных рекомендаций.

In [11]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2)) 

0.78 0.62


In [12]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}") 

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


In [13]:
# сохрянаяем важные переменные
# events_test.to_parquet('cache/events_test.parquet')
# events_train.to_parquet('cache/events_train.parquet')

In [14]:
# Приборка
for var_name in (    
    'cold_user_items_no_avg_rating_idx',
    'cold_user_recs',
    'cold_users',
    'cold_users_events_with_recs',
    'cold_users_hit_ratio',
    'common_users',
    'item_popularity',
    'rmse',
    'mae',
    'top_k_pop_items',
    'users_test',
    'users_train'
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

3.5 Gb

### Тема 2 Урок 5 Матрица взаимодействий и первые персональные рекомендации

Реализация SVD-алгоритма    
Воспользуемся готовой реализацией SVD-алгоритма из библиотеки surprise. В качестве разбиения данных на train и test возьмём разбиение из предыдущего урока: events_train, events_test.

Получение рекомендации

In [15]:
# Приборка
for var_name in (    
    'reader', 'surprise_train_set', 'svd_model', 'surprise_test_set', 'svd_predictions', 'rmse', 'mae', 'random_model', 'random_predictions', 'user_id', 'user_history', 'user_history_to_print', 'user_recommendations'
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

### Тема 3 Урок 2 Коллаборативная фильтрация: ALS

In [16]:
events = pd.read_parquet('goodsread/events.parquet')

In [17]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

In [18]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

Имея подготовленную матрицу взаимодействий, перейдём к третьему шагу — создадим ALS-модель. Для примера возьмём количество латентных факторов для матриц $P, Q$, равным 50.      
Выполните код для создания и тренировки модели.

In [19]:
from implicit.als import AlternatingLeastSquares
from threadpoolctl import threadpool_limits

threadpool_limits(1, "blas")

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 50/50 [03:00<00:00,  3.62s/it]


Чтобы получить рекомендации для пользователя с помощью модели ALS, используем такую функцию:

In [20]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

Получим рекомендации для всех имеющихся пользователей. Выполните код ниже.

In [21]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100
)

Код возвращает рекомендации как список списков, это не очень удобно. Преобразуем его в более удобный формат — табличный.

In [22]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

Сохраним полученные рекомендации в файл, они ещё нам пригодятся.

In [23]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("cache/als_recommendations.parquet")

На практике часто используют метрику NDCG, она принимает значение от 0 (предлагаемый порядок никак не соответствует истинному) до 1 (предлагаемый порядок в точности соответствует истинному). Подробнее о NDCG можно прочитать на MachineLearningInterview.com.     
Для удобства оценки добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки:

In [24]:
als_recommendations = (
    als_recommendations
    .merge(
        events_test[["user_id", "item_id", "rating"]].rename(columns={"rating": "rating_test"}), 
        on=["user_id", "item_id"],
        how="left"
    )
)

Подсчитать метрику NDCG для одного пользователя поможет готовая реализация из scikit-learn:

In [25]:
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

Умея считать NDCG для одного пользователя, посчитаем данную метрику, например, для
k=5 для всех пользователей из тестовой выборки. В результате каждому пользователю будет соответствовать одно значение NDCG@5. 

In [26]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(
    lambda x: compute_ndcg(x["rating_test"], x["score"], k=5)
)

Имея ряд значений NDCG@5 по пользователям, можно посчитать среднее её значение (по всем пользователям):

In [27]:
print(ndcg_at_5_scores.mean())

0.975946709792109


In [28]:
# Приборка
for var_name in (    
    'events', 'user_encoder', 'item_encoder', 'user_item_matrix_train', 'als_model', 'als_recommendations',
    'user_ids_encoded', 'item_ids_enc', 'als_scores', 'rating_test_idx', 'ndcg_at_5_scores'
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

### Тема 3 Урок 3 Контентные рекомендации

Сначала преобразуем значения в genre_and_votes из текстового представления в тип в Python:

In [29]:
# items_loaded = pd.read_parquet('goodsread/items.parquet')

In [30]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

Задание 1 из 4     
Теперь составьте список жанров с долями голосов по ним в genres.  Дополните и выполните следующий код:

In [31]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v['genre_and_votes']
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = votes

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

Результат выполнения кода — список жанров с долями голосов по ним в genres. Посмотрим на самые популярные жанры:

In [32]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850115,0.149498
1,Fiction,6406698,0.139821
38,Classics,3415071,0.074531
18,Young Adult,3297027,0.071955
34,Romance,2422690,0.052873
5,Nonfiction,1737798,0.037926
16,Historical-Historical Fiction,1531489,0.033423
20,Mystery,1371370,0.029929
24,Science Fiction,1218997,0.026604


Задание 2 из 4    
Функция в коде ниже строит матрицу вида «книга-жанр». Изучите её. Подумайте, что будет соответствовать столбцам матрицы. 

In [33]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1 (сумма по строке = 1)
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr 

Получим матрицу с весами по жанрам для каждой книги:

In [34]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Аналогичным образом получим матрицу с весами по жанрам для какого-нибудь пользователя, например, для пользователя с идентификатором 1000010.    
Задание 3 из 4     
Дополните и выполните код ниже, чтобы получить описанную матрицу. 

In [35]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)
user_items_genres_csr

<22x815 sparse matrix of type '<class 'numpy.float64'>'
	with 149 stored elements in Compressed Sparse Row format>

На практике часто пользователь явно указывает предпочтения в своём профиле. У нас таких данных нет, поэтому предпочтения пользователя по жанрам вычислим автоматически на основе его истории поведения. 

In [36]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0)) 

Можно посмотреть, какие жанры больше всего нравятся пользователю:

In [37]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5) 

,name,votes,score
genre_id,,,
1,Fiction,6406698,0.185241
38,Classics,3415071,0.103879
25,Fantasy,6850115,0.072447
5,Nonfiction,1737798,0.050865
24,Science Fiction,1218997,0.040920


Построим рекомендации       
Теперь рассчитаем рекомендации на основе двух объектов:      
all_items_genres_csr — матрица распределения интересов всех пользователей по жанрам. Строка — вектор распределения интересов всех пользователей по жанрам для одного объекта.
user_genres_scores — вектор интересов пользователя по жанрам.
Рекомендации будем рассчитывать с помощью косинусного сходства между двумя векторами:     
 
Чтобы посчитать косинусное сходство user_genres_scores к вектор-строкам из all_items_genres_csr, необходимо выполнить вычисления по формуле столько раз, сколько есть строк в all_items_genres_csr.
Задание 4 из 4
Получите наиболее релевантные рекомендации для пользователя. Дополните код так, чтобы переменная top_k_indices заполнялась индексами соответствующих книг. Для этого удобно использовать np.argsort от similarity_scores, подсчитанной для всех книг.
После вычисления top_k_indices по полученным индексам извлеките список объектов, которые могут быть интересны пользователю, при помощи кода:

In [38]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[:k]

In [39]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]])

,author,title,genre_and_votes
1673075,Mo Willems,"Today I Will Fly! (Elephant & Piggie, #1)","{'Childrens-Picture Books': 409, 'Childrens': 199}"
1523576,Lester Sumrall,Gifts and Ministries of the Holy Spirit,"{'Christian': 6, 'Religion-Theology': 3, 'Christian-Christian Non Fiction': 2, 'Spirituality': 2}"
1422146,Nicholas Davies,Diana the Killing of a Princess,None
515584,Nayyirah Waheed,Nejma,"{'Poetry': 929, 'Feminism': 31}"
956516,"Tina Marie Kaht, Valeria Avantario",Grandparents' Day,{'Childrens': 2}


In [40]:
# Приборка
for var_name in (    
    'user_id', 'user_events', 'user_items', 'user_items_genres_csr', 
    'user_ratings', 'user_items_genres_weighted', 'user_genres_scores', 'user_genres', 'similarity_scores',
    'top_k_indices', 'selected_items', 'k'
    # 'genres', 'all_items_genres_csr',
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

### Тема 3 Урок 4 Валидация

In [41]:
als_recommendations = pd.read_parquet('cache/als_recommendations.parquet')

Посчитаем recall и precision для ALS-рекомендаций (als_recommendations). Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
gt (ground truth): объект есть в тестовой выборке;
pr (predicted): объект есть в рекомендациях.
Теперь разметим признаки бинарной классификации:
TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

In [42]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common 

Обработаем ALS-рекомендации для подсчёта метрик для 5 лучших рекомендаций:

In [43]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=5
)

Common users: 123223


Дополните код функции compute_cls_metrics для расчёта recall. Получите значения метрик precision@5, recall@5.

In [44]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [45]:
compute_cls_metrics(events_recs_for_binary_metrics)

(0.007581376853347184, 0.014121568795222568)

In [46]:
# Приборка
for var_name in (    
    'events_recs_for_binary_metrics', 
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

### Тема 4 Урок 2 Специфичные метрики

Задание 1 из 2  
Для рекомендаций, сохранённых в переменной als_recommendations, посчитайте покрытие по объектам согласно формуле выше. При этом используйте весь топ-100 рекомендаций.

In [47]:
# расчёт покрытия по объектам
cov_items = als_recommendations['item_id'].nunique() / len(items)
print(f"{cov_items:.2f}") 

0.09


Задание 2 из 2  
Посчитайте среднее Novelty@5 для als_recommendations. Для этого: 
разметьте каждую рекомендацию в als_recommendations булевым признаком read (False — пользователь не читал книгу, True — пользователь читал книгу), используя events_train,
посчитайте Novelty@5 для каждого пользователя,
посчитайте среднеарифметическое для полученных значений Novelty@5.

In [48]:
als_recommendations.head(1)

,user_id,item_id,score
0,1000000,3,0.990941


In [49]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(
    events_train[["read", "user_id", "item_id"]], 
    on=["user_id", "item_id"],
    how="left"
)
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by=['user_id', 'score'], ascending=[True, False])
als_recommendations["rank"] = als_recommendations.groupby('user_id').cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
novelty_5.mean()

0.607333279143491

In [50]:
# Приборка
for var_name in (    
    'cov_items', 'novelty_5'
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

### Тема 4 Урок 3

Задание 1 из 6  
Используем отложенную тестовую часть данных — назовём её events_test — для получения двух новых частей данных:
одна, составляющая первые 45 дней, будет использоваться для таргетов,
другая, состоящая из 45 последних дней, будет новой тестовой выборкой.
Завершите код так, чтобы в events_labels оказалась первая часть данных, а в events_test_2 — вторая.

In [66]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy() 

Задание 2 из 6  
Объедините имеющихся кандидатов по совпадению user_id, item_id в один список.

In [67]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
) 

Добавим в candidates таргеты, используя историю взаимодействий в events_labels. 
Задание 3 из 6  
Дополните код ниже.
В candidates добавьте колонку target со значениями:
1 для тех item_id, которые пользователь прочитал (положительный пример).
0 — для всех остальных (негативный пример).
В candidates_for_train отберите все положительные примеры, а также не менее четырёх негативных примеров для каждого пользователя в положительных примерах.

In [68]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], how='left')
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query('target == 1'),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

Обучение модели  
Выше мы подготовили все нужные артефакты для обучения модели ранжирования. Выполните код, чтобы обучить модель.

In [69]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 21.5ms	remaining: 21.5s
100:	learn: 0.5118959	total: 1.88s	remaining: 16.8s
200:	learn: 0.5111710	total: 3.86s	remaining: 15.4s
300:	learn: 0.5105208	total: 5.83s	remaining: 13.5s
400:	learn: 0.5100174	total: 7.78s	remaining: 11.6s
500:	learn: 0.5095747	total: 9.77s	remaining: 9.73s
600:	learn: 0.5091600	total: 11.7s	remaining: 7.79s
700:	learn: 0.5087803	total: 13.7s	remaining: 5.86s
800:	learn: 0.5084220	total: 15.7s	remaining: 3.91s
900:	learn: 0.5080930	total: 17.7s	remaining: 1.94s
999:	learn: 0.5078081	total: 19.6s	remaining: 0us


Задание 4 из 6   
Дополните код ниже так, чтобы в candidates_to_rank попали кандидаты от обоих базовых генераторов подобно тому, как это было сделано для фазы тренировки выше.

In [70]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(
    als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer"
)

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test["user_id"].drop_duplicates())]
print(len(candidates_to_rank)) 

23830721


Задание 5 из 6      
Дополните код для того, чтобы вызвать модель и получить для каждого пользователя топ-100 рекомендаций — значение rank нужно выставить не более ста.

In [71]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id')['cb_score'].cumcount(ascending=False) + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query('rank <= @max_recommendations_per_user')

Задание 6 из 6
Посчитайте метрики recall и precision.
Используйте полученные рекомендации final_recommendations, отложенную тестовую выборку events_test_2, созданные в уроке «Валидация» предыдущей темы.
А также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [72]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.002, recall: 0.005


In [73]:
# Приборка
for var_name in (    
    'split_date_for_labels','split_date_for_labels_idx', 'events_test_2', 
    'als_recommendations', 'content_recommendations', 'candidates', 'candidates_to_sample',
    'negatives_per_user', 'features', 'target', 'train_data', 'cb_model',
    'als_recommendations_2', 'content_recommendations_2', 'inference_data',
    'predictions', 'max_recommendations_per_user', 'final_recommendations',
    'cb_events_recs_for_binary_metrics_5', 'cb_precision_5', 'cb_recall_5'
    # 'events_inference', 'candidates_for_train', 'candidates_to_rank', 'events_labels'
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

### Тема 4 Урок 4 Как использовать признаки для улучшения рекомендаций

Задание 1 из 6      
Посчитаем новый признак — «возраст» книги на основе года публикации. Назовём его age. 
Используя обновлённый справочник объектов items, добавьте признаки возраста age и средней популярности average_rating к кандидатам для тренировки модели candidates_for_train и к кандидатам для ранжирования candidates_to_rank.

In [74]:
items["age"] = 2018-items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

candidates_for_train = candidates_for_train.merge(
    items[['item_id', 'age', 'average_rating']],
    on='item_id',
    how='left'
)
candidates_to_rank = candidates_to_rank.merge(
    items[['item_id', 'age', 'average_rating']],
    on='item_id',
    how='left'
)

Задание 2 из 6  
Используя события в events_train и events_inference, посчитайте и добавьте признаки пользователей к кандидатам в candidates_for_train и candidates_to_rank соответственно:
reading_years — длительность истории пользователя,
books_read — количество книг, прочитанных за всё время,
books_per_year — среднее количество прочитанных книг в год,
rating_avg — средняя оценка,
rating_std — дисперсия оценок.

In [75]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max()-x.min()).days/365.25 if (x.max()-x.min()).days else 1/365.25),
        books_read=('read', 'sum'),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std"))
    
    # display(
    #     (user_features["reading_years"] == 0).value_counts()
    # )
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features

user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")

# оставим только тех пользователей, что есть в тесте, для экономии ресурсов
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

user_features_for_ranking = get_user_features(events_inference)
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")

Задание 3 из 6      
Используя истории events_train и events_inference, а также ранее полученные артефакты по жанрам книг — словарь жанров genres, оценки книг по жанрам all_items_genres_csr — добавьте парные признаки, по одному на каждый жанр, которые совместно показывают, какие жанры предпочитает пользователь. 
Жанровость в данном случае — численный коэффициент принадлежности книги к жанру. Например, если пользователь прочитал три книги, которые с весами 0.3, 0.2, 0.4 из  all_items_genres_csr относятся к Fantasy, то интерес пользователя к Fantasy составляет среднее этих трёх оценок — 0.3.
Для экономии ресурсов возьмём не все жанры, а десять наиболее популярных. Все остальные отметим как не вошедшие в топ и обозначим как others. 

In [77]:
# Приборка
for var_name in (    
    'invalid_age_idx', 'user_features_for_train', 'user_features_for_ranking'
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name

### Тема x Урок yy

In [78]:
# Приборка
for var_name in (    
    '',
):
    if locals().get(var_name) is not None:
        del locals()[var_name]
del var_name